# Simplest Meaningful Neural Net

After Andrew Trask's [Simple Neural Network](http://iamtrask.github.io/2015/07/12/basic-python-network/).

***Note on `nonlin()`***

The `deriv=` parameter in the original `nonlin()` function is _extremely misleading_. The forward-pass nonlinear activation function is the _sigmoid_ $\sigma(x)$. The dot product of the outputs of the previous stage/layer with the weights between the previous and current stage/layer is passed through the non-linear function. The backward pass applies the derivative of the non-linear activation function to the same dot product to calculate the error term $\delta$ for the current layer. The drivative of the sigmoid is $\sigma'(x) = \frac{d\sigma}{dx} = \sigma(x)(1 - \sigma(x))$.

`nonlin()` returns `x * (1 - x)` when `deriv=True`, which is *incorrect* and only works because the the _post-nonlinearity output_ is used as an argument (**l2** for the output layer and **l1** for the hidden layer), that is, the _sigmoid_ is already applied to it. This will work only in this case where the derivative is a straighforward expression involving the original funcion.

The sigmoid derivative expression can be used to save calculations, but in the interest of clarity over efficiency for this exposition, the two expressions are separated. It is important that _the derivative of the nonlinear activation function **is applied to the dot-product input of the node** and not its output_. This is in line with general computational graph theory and best practices.

This change, of course, will percolate and will require additional changes in notation. The layer values **l0**, **l1**, and **l2** have been divided into **lj_in** and **lj_out** to represent the input to the node at level **j** and the (post-nonlineartiy) output of the node at level **j**. **l0**, which represents the input **X**, corresponds to **l0_out** in this scheme.

## Overview

This is a 2-layer neural net (3-layer, if we count the input nodes as a layer) with 3 input nodes, one output node, and a four-node hidden layer. There are two layers (sets) of weights:
- between input nodes and hidden nodes
- between hidden nodes and output node
The _sigmoid_ function is used for nonlinearity.

In [218]:
import numpy as np

def nonlin(x, deriv=False):
    def sigmoid(x):
        return 1 / (1 + np.exp( -x ))
    s = sigmoid(x)
    if(deriv == True):
        return s * (1 - s)
    else:
        return s

## Input

The input sample are arranged as _rows_. There are _4 input samples_. Each sample is _a row of 3 input values_. This results in a 4 x 3 matrix.

In [219]:
# Input samples
X = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1]])

In [220]:
X.shape

(4, 3)

In [221]:
X

array([[0, 0, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 1]])

The outputs forms a _column vector_ since we only have _one output node_.

In [222]:
# Training target
y = np.array([[0],
              [1],
              [1],
              [0]])

In [223]:
y.shape

(4, 1)

In [224]:
y

array([[0],
       [1],
       [1],
       [0]])

## Weights

The two sets of weights are initialized randomly with mean zero. 

### Input to Hidden

The weights between the 3 input nodes and the 4 hidden-layer nodes is a 3 x 4 matrix. Each row contains the weights from one input node to the hidden nodes.

In [225]:
np.random.seed(1)

# Randomly initialize our weights with mean 0
syn0 = 2 * np.random.random((3, 4)) - 1

In [226]:
syn0.shape

(3, 4)

In [227]:
syn0

array([[-0.16595599,  0.44064899, -0.99977125, -0.39533485],
       [-0.70648822, -0.81532281, -0.62747958, -0.30887855],
       [-0.20646505,  0.07763347, -0.16161097,  0.370439  ]])

The dot product of the input **X** and the weights **syn0** can be computed _without transposition_. It is a (4 x 3) • (3 x 4) = (4 x 4) matrix. This corresponds to **l1_in** in the code, the input to the hidden layer (layer 1).

In [228]:
l1_in = np.dot(X, syn0)

In [229]:
l1_in

array([[-0.20646505,  0.07763347, -0.16161097,  0.370439  ],
       [-0.91295327, -0.73768934, -0.78909055,  0.06156045],
       [-0.37242104,  0.51828245, -1.16138222, -0.02489585],
       [-1.07890926, -0.29704036, -1.7888618 , -0.3337744 ]])

Each position in this 4 x 4 matrix is the dot product of one input sample (3 values) with the weights to one hidden node (3 values), and represents the _pre-nonlinearity_ value of that hidden-layer node. The _ij_ position of the matrix corresponds to the _pre-nonlinearity_ value of the _j-th_ node for the _i-th_ sample.

In [230]:
np.dot(X[0, :], syn0[:, 0])  # The pre-nonlinearity value of the 0-th hidden node for the 0-th sample.

-0.20646505153866013

In [231]:
np.dot(X[3, :], syn0[:, 2])  # The pre-nonlinearity value of the 2-nd hidden node for the 3-rd sample.

-1.7888617988033788

One row of the 4 x 4 matrix represents the _pre-nonlinearity_ values for all the hidden nodes. The _i-th_ row represents the _pre-nonlinearity_ value/output of the hidden layer for the _i-th_ training input sample.

In [232]:
np.dot(X[3, :], syn0)  # The pre-nonlinearity value of the hidden layer for the 3-rd sample.

array([-1.07890926, -0.29704036, -1.7888618 , -0.3337744 ])

The _post-nonlinearity_ output of the hidden layer for the _i-th_ sample is also a 1 x 4 matrix (or, almost equivalently, a vector of length 4). The non-linear _sigmoid_ is applied _elementwise_ on the vector. _**Notice the values are all between 0.0 and 1.0.**_

In [233]:
nonlin(np.dot(X[3, :], syn0))

array([ 0.25371248,  0.42628115,  0.14321233,  0.41732254])

The _post-nonlinearity_ output of the hidden layer for the the sample matrix is also a 4 x 4 matrix, with each row representing the _post-nonlinearity_ output of the hidden layer for the corresponding input sample. This corresponds to **l1_out** in the code.

In [234]:
l1_out = nonlin(l1_in)

In [235]:
l1_out 

array([[ 0.44856632,  0.51939863,  0.45968497,  0.59156505],
       [ 0.28639589,  0.32350963,  0.31236398,  0.51538526],
       [ 0.40795614,  0.62674606,  0.23841622,  0.49377636],
       [ 0.25371248,  0.42628115,  0.14321233,  0.41732254]])

### Hidden to Output

The weights between the 4 hidden-layer nodes and the output node is a 4 x 1 matrix. There are 4 hidden-layer nodes that feed into the output node.

In [236]:
syn1 = 2 * np.random.random((4, 1)) - 1

In [237]:
syn1.shape

(4, 1)

In [238]:
syn1

array([[-0.5910955 ],
       [ 0.75623487],
       [-0.94522481],
       [ 0.34093502]])

The hidden nodes emit their _post-nonlinearity_ output to the output node and are multiplied by the hidden-to-output weights. The _pre-nonlinearity_ output of the output node for the _i-th_ input sample is the _i-th_ 1-element row of the _pre-nonlinearity_ output matrix. 

In [239]:
np.dot(nonlin(np.dot(X[3, :], syn0)), syn1)  # Pre-nonlinearity output for the 3-rd input sample

array([ 0.17931239])

In [240]:
np.dot(nonlin(np.dot(X, syn0)), syn1)  # Pre-nonlinearity 4 x 1 output matrix (4 input samples, 1 output per sample)

array([[-0.10517857],
       [-0.04417936],
       [ 0.17581491],
       [ 0.17931239]])

## Forward Pass

The non-linear _sigmoid_ is applied _elementwise_ to the _pre-nonlinearity_ output matrix. The following line represents the complete _forward pass_ through the neural network. The matrix multiplications are as follows: (4 x 3) • (3 x 4) • (4 x 1) = (4 x 1). This is called **l2_out** in the code.

In [241]:
l2_in = nonlin(np.dot(l1_out, syn1))
l2_out = nonlin(l2_in)
# l2_out = nonlin(np.dot(nonlin(np.dot(X, syn0)), syn1))  # The complete forward pass in one line

In [242]:
l2_out

array([[ 0.61626612],
       [ 0.61986069],
       [ 0.63270545],
       [ 0.63290702]])

## Backward Pass

### Error

The _backward_ (aka _backpropagation_) pass starts with computing the error of the _target_ and _output_ matrices (the **l2_error** matrix), and proceeds to propagate that error back along the network, guided by the local gradient at each node.

In [243]:
l2_error = y - l2_out  # Error matrix. Each row is the error on the corresponding traning sample (X, y)

In [244]:
l2_error

array([[-0.61626612],
       [ 0.38013931],
       [ 0.36729455],
       [-0.63290702]])

For backpropagation, the output error needs to be _scaled by the derivative of the nonlinear function, **evaluated at the input to the node/layer**_. The _derivative matrix_ for the nonlinearity contains the sigmoid derivative evaluated at the input from the previous layer, for each sample.

In [245]:
nonlin(l2_in, deriv=True)

array([[ 0.23648219],
       [ 0.23563342],
       [ 0.23238926],
       [ 0.23233572]])

### Error Term

The _layer 2 error term_ is the error, scaled by the derivative. It is denoted as $\delta_{l2}$.

In [246]:
l2_delta = l2_error * nonlin(l2_in, deriv=True)

In [247]:
l2_delta.shape  # The error term has the same shape as the output of the layer

(4, 1)

In [248]:
l2_delta

array([[-0.14573596],
       [ 0.08957352],
       [ 0.08535531],
       [-0.14704691]])

The _error term_ for a layer is used for two operations (in **this order**):

1. **Propagate the error to the preceding layer.** Propagating the error backward to a preceding layer (that is, caculating the preceding layer's contribution to the error) requires the error term of the next layer and the weights between the previous and the next. _**It is important to propagate the error BEFORE updating the weights. Otherwise the error calculation will be incorrect.**
2. **Update the weights preceding the layer.** The weights scale the output of the _previous layer_ and contribute to the output error of the next level through the nonlinear activation function. In backpropagation, they are updated by adding to the weights the output from the previous layer, scaled by the error term for the next layer. This nudges the weights in the direction opposite their error. The update can be optionally scaled by a _learning rate_ $\eta$ in the interval $(0.0, 1.0]$.

The _l2 error term_ `l2_delta` will be used to:

1. Calculate the error for the precding layer **l1**.
2. Update the weights between the hidden and output layers, **syn1**. 

#### Error (Back-) Propagation

The error for a layer has to have the same shape as its output. **l1_out** is a 4 x 4 matrix so the error matrix also has to be 4 x 4. We have to distribute conributions from 4 hidden-layer nodes for 4 input samples. For each sample, we need to distribute the error to all hidden-layer nodes for a 4 x 4 matrix. Since the hidden-to-output weight matrix is 4 x 1, we need to transpose it before taking the dot product with the layer 2 error term.

In [249]:
l2_delta.shape, syn1.shape, l1_out.shape

((4, 1), (4, 1), (4, 4))

In [250]:
l1_error = l2_delta.dot(syn1.T)

In [251]:
l1_error.shape 

(4, 4)

In [252]:
l1_error  # For each sample, we know how much each node contributes to the error

array([[ 0.08614387, -0.11021062,  0.13775325, -0.04968649],
       [-0.05294651,  0.06773862, -0.08466712,  0.03053875],
       [-0.05045314,  0.06454866, -0.08067996,  0.02910061],
       [ 0.08691877, -0.111202  ,  0.13899239, -0.05013344]])

`l1_error` allows us to compute the the _layer 1 error term_ the same way we calculated the _layer 2 error term_ above. `l1_delta` will allow us to update the input-to-hidden weights. Since we have no more layers, there is no more error propagation to do, so we can proceed to updating the weights.

In [253]:
l1_delta = l1_error * nonlin(l1_in, deriv=True)

In [254]:
l1_delta

array([[ 0.02130808, -0.02751118,  0.03421442, -0.01200504],
       [-0.01082085,  0.01482468, -0.01818588,  0.00762746],
       [-0.01218584,  0.01510022, -0.01464938,  0.00727403],
       [ 0.01645741, -0.02719618,  0.01705472, -0.01219067]])

#### Weight Update

Since we are adding to the weight matrix, we need a 4 x 1 update matrix. The **l1_out** is a 4 x 4 matrix and the **l2_delta** matrix is 4 x 1. (4 x 4) • (4 x 1) = (4 x 1). However, we have to make sure that we are constructing and update matrix where each weight is updated by a value that captures its contributions to all the sample errors. Since each row of **l1_out** represents the output of layer 1 for the corresponding input sample, we cannot take the dot product **l2_out** • **l2_delta**, because each resulting value will be a combination of contributions of the nodes of the whole layer. To isolate the contributions of the node we are updating, we need to _transpose_ the output matrix before we use it in the dot product. Thus, each row will represent the contributions from the corresponding node.

In [255]:
syn1.shape, l1_out.shape, l2_delta.shape

((4, 1), (4, 4), (4, 1))

In [256]:
l1_out.T.dot(l2_delta)  # Proper way to calculate the update matrix (see explanation above)

array([[-0.04220517],
       [-0.05590438],
       [-0.03972193],
       [-0.05926698]])

In [257]:
syn1 += l1_out.T.dot(l2_delta)

In [258]:
syn1

array([[-0.63330067],
       [ 0.70033049],
       [-0.98494674],
       [ 0.28166804]])

The update of the input-to-hidden weights proceeds accordingly.

In [259]:
syn0.shape, l0_out.shape, l1_delta.shape

((3, 4), (4, 3), (4, 4))

In [260]:
l0_out.T.dot(l1_delta)  # We need 3 sets of 4 weight updates, so the input (4 samples of 3 inputs) has to be transposed

array([[ 0.00427157, -0.01209596,  0.00240534, -0.00491664],
       [ 0.00563656, -0.0123715 , -0.00113116, -0.00456321],
       [ 0.0147588 , -0.02478246,  0.01843388, -0.00929423]])

In [261]:
syn0 += l0_out.T.dot(l1_delta)

In [262]:
syn0

array([[-0.16168442,  0.42855303, -0.99736591, -0.4002515 ],
       [-0.70085166, -0.82769431, -0.62861074, -0.31344176],
       [-0.19170625,  0.05285101, -0.14317709,  0.36114477]])

## Training Epoch

Training consists of an _epoch loop_, containing:

- a single forward pass
- a single backward pass
- a weight update

Training proceeds over some large number of epochs. By gradient descent, the small updates to the weights eventually reduce the error to beneath some target threshold.

In [263]:
import numpy as np

def nonlin(x, deriv=False):
    def sigmoid(x):
        return 1 / (1 + np.exp( -x ))
    s = sigmoid(x)
    if deriv == True:
        return s * (1 - s)
    else:
        return s
    
# Inputs
X = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1]])

# Targets
y = np.array([[0],
              [1],
              [1],
              [0]])

# Weights
np.random.seed(1)

# Randomly initialize our weights with mean 0
syn0 = 2 * np.random.random((3, 4)) - 1
syn1 = 2 * np.random.random((4, 1)) - 1

# Train
for j in range(60000):

    # I. FORWARD PASS
    # Feed forward through layers 0, 1, and 2
    l0_out = X
    l1_in = np.dot(l0_out, syn0)
    l1_out = nonlin(l1_in)
    l2_in = np.dot(l1_out, syn1)
    l2_out = nonlin(l2_in)

    # How much did we miss the target value?
    l2_error = y - l2_out
    
    if (j % 10000) == 0:
        print("Error: {}".format(str(np.mean(np.abs(l2_error)))))
        
    # II. BACKWARD PASS
    # In what direction is the target value?
    # Were we really sure? If so, don't change too much.
    l2_delta = l2_error * nonlin(l2_in, deriv=True)

    # how much did each l1 value contribute to the l2 error (according to the weights)?
    l1_error = l2_delta.dot(syn1.T)
    
    # in what direction is the target l1?
    # were we really sure? if so, don't change too much.
    l1_delta = l1_error * nonlin(l1_in, deriv=True)

    # III. WEIGHT UPDATE
    syn1 += l1_out.T.dot(l2_delta)
    syn0 += l0_out.T.dot(l1_delta)


Error: 0.496410031903
Error: 0.00858452565325
Error: 0.00578945986251
Error: 0.00462917677677
Error: 0.00395876528027
Error: 0.00351012256786
